This notebook shows how a sound model works.

In [ ]:
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt

import os
from utils.physics.sound_model.spherical_sound_model import HomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.physics.sound_model.ellipsoidal_sound_model import GridEllipsoidalSoundModel as GridSoundModel
from utils.physics.geodesic.distance import distance_point_point

Propagation time computation

In [ ]:
# propagation time

pos_1 = [-24.2053, 63.0102]  # lat, lon format (in degrees)
pos_2 = [-31.5758, 83.2423]
# note: to get the position of a station s, just do s.get_pos()

# uncomment the wanted sound model
# sound_model = GridSoundModel([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)], loader='netcdf')

ISAS_PATH = "/media/rsafran/CORSAIR/ISAS/extracted/2018"
arr = os.listdir(ISAS_PATH)
file_list = [os.path.join(ISAS_PATH, fname) for fname in arr if fname.endswith('.nc')]
sound_model = GridSoundModel(file_list, loader='ISAS')

# sound_model = HomogeneousSoundModel(sound_speed=1480)


propagation_time = sound_model.get_sound_travel_time(pos_1, pos_2, datetime.now())
print(f"Propagation time between {pos_1} and {pos_2}: {propagation_time:.2f}s")
print(f"(Distance is {distance_point_point(pos_1, pos_2)/1_000:.2f} km)")

Localization (given 4 detection time and positions, find the source)

In [ ]:
# location
detection_pos_1 = [2, 78]
detection_pos_2 = [2, 82]
detection_pos_3 = [-2, 79]
detection_pos_4 = [-2, 82]
detection_positions = [detection_pos_1, detection_pos_2, detection_pos_3, detection_pos_4]

detection_time_1 = datetime(2020, 1, 1, 0, 0, 0)
detection_time_2 = detection_time_1
detection_time_3 = detection_time_1
detection_time_4 = detection_time_1
detection_times = [detection_time_1, detection_time_2, detection_time_3, detection_time_4]

inversion = sound_model.localize_common_source(detection_positions, detection_times,initial_pos =[-1,0.0,80] )
print(f"Found source position {inversion.x[1:]} with an emission {inversion.x[0]:.2f} s before the first detection. Cost was {inversion.cost:.2f}.")

In [ ]:
# # location
# detection_pos_1 = [2, 0]
# detection_pos_2 = [-2, 0]
# detection_pos_3 = [0, 2]
# detection_pos_4 = [0, -2]
# detection_positions = [detection_pos_1, detection_pos_2, detection_pos_3, detection_pos_4]
#
# detection_time_1 = datetime(2020, 1, 1, 0, 0, 0)
# detection_time_2 = detection_time_1
# detection_time_3 = detection_time_1
# detection_time_4 = detection_time_1
# detection_times = [detection_time_1, detection_time_2, detection_time_3, detection_time_4]

inversion, weights, uncertainties = sound_model.localize_with_uncertainties(detection_positions, detection_times,initial_pos =[-1,0.02,80], drift_uncertainties=[2,1,1,1], pick_uncertainties=[2,1,1,1],velocity_uncertainties=[2,1,1,1] )
print(f"Found source position {inversion.x[1:]} with an emission {inversion.x[0]:.2f} s before the first detection. Cost was {inversion.cost:.2f}.")

In [ ]:
weights

Now do this same process but adding random noise on both detection times and sound velocities, to obtain a Monte-Carlo sampling of the position.

In [ ]:
# MC uncertainty estimate
pos = sound_model.MC_error_estimation(1000, pick_sigma=1.5, velocity_sigma=0.5, sensors_positions=detection_positions, detection_times=detection_times)
pos = np.array(pos)
plt.scatter(pos[:,2], pos[:,1])

In [ ]:
import datetime
import numpy as np
from utils.physics.sound_model.ellipsoidal_sound_model import EllipsoidalSoundModel, HomogeneousEllipsoidalSoundModel
model  =sound_model
def test_tdoa_localization():
    """Test TDOA localization with known source position"""
    
    # Create model with constant sound speed
    sound_speed = 1485.5  # m/s
    model = HomogeneousEllipsoidalSoundModel(sound_speed)
    
    # Test case 1: Source at known position
    print("=== Test 1: Source décalée ===")
    source_true = [2.5, -4.0]  # lat, lon
    
    # Sensor positions
    sensors = [
        [5, 0],    # Sensor 0 (reference)
        [-5, 0],   # Sensor 1  
        [0, 5],    # Sensor 2
        [0, -5]    # Sensor 3
    ]
    
    # Calculate true travel times
    base_time = datetime.datetime(2020, 1, 1, 0, 0, 0)
    detection_times = []
    
    # print(f"Source vraie: {source_true}")
    # print("Calcul des temps de détection théoriques:")
    
    for i, sensor in enumerate(sensors):
        distance = model.get_distance(source_true, sensor)
        travel_time = distance / sound_speed
        detection_time = base_time + datetime.timedelta(seconds=travel_time)
        detection_times.append(detection_time)
        
        print(f"  Capteur {i} à {sensor}: distance={distance:.1f}m, temps_vol={travel_time:.3f}s")
    
    # Localize
    result = model.localize_common_source(sensors, detection_times)
    print(result)
    print(f"\nRésultat inversion:")
    print(f"  Position trouvée: [{result.x[1]:.6f}, {result.x[2]:.6f}]")
    print(f"  Position vraie:   [{source_true[0]:.6f}, {source_true[1]:.6f}]")
    print(f"  Erreur: {np.linalg.norm(np.array(result.x[1:]) - np.array(source_true)):.6f}")
    print(f"  Coût final: {result.cost:.6f}")
    print(f"  Succès: {result.success}")
    
    # Test case 2: Equidistant source (your original case)
    print("\n=== Test 2: Source équidistante (centre) ===")
    source_center = [0.0, 0.0]
    
    # All sensors detect at the same time (equidistant)
    detection_times_center = [base_time] * 4
    
    result_center = model.localize_common_source(sensors, detection_times_center)
    
    print(f"Position trouvée: [{result_center.x[1]:.6f}, {result_center.x[2]:.6f}]")
    print(f"Position vraie:   [{source_center[0]:.6f}, {source_center[1]:.6f}]")
    print(f"Erreur: {np.linalg.norm(np.array(result_center.x[1:]) - np.array(source_center)):.6f}")
    print(f"Coût final: {result_center.cost:.6f}")
    
    # Test case 3: Source with realistic time differences
    print("\n=== Test 3: Source avec différences de temps réalistes ===")
    source_realistic = [1.0, 0.5]
    
    # Add some realistic time differences (in seconds)
    time_offsets = [0.0, 0.01, -0.05, 0.08]  # seconds
    detection_times_realistic = []
    
    for i, (sensor, offset) in enumerate(zip(sensors, time_offsets)):
        distance = model.get_distance(source_realistic, sensor)
        travel_time = distance / sound_speed + offset
        detection_time = base_time + datetime.timedelta(seconds=travel_time)
        detection_times_realistic.append(detection_time)
        print(f"  Capteur {i}: distance_théo={model.get_distance(source_realistic, sensor):.1f}m, offset={offset}s")
    
    result_realistic = model.localize_common_source(sensors, detection_times_realistic)
    
    print(f"\nPosition trouvée: [{result_realistic.x[1]:.6f}, {result_realistic.x[2]:.6f}]")
    print(f"Position vraie:   [{source_realistic[0]:.6f}, {source_realistic[1]:.6f}]")
    print(f"Erreur: {np.linalg.norm(np.array(result_realistic.x[1:]) - np.array(source_realistic)):.6f}")
    print(f"Coût final: {result_realistic.cost:.6f}")


def compare_with_spherical():
    """Compare TDOA ellipsoidal vs spherical models"""
    print("\n=== Comparaison Ellipsoïdal vs Sphérique ===")
    
    # Large distance test to see difference
    source = [45.0, 2.0]  # Bretagne, France
    sensors = [
        [44.0, 1.0],   # Sud-Ouest
        [46.0, 1.0],   # Nord-Ouest  
        [45.0, 0.0],   # Ouest
        [45.0, 4.0]    # Est
    ]
    
    sound_speed = 1485.
    ellip_model = HomogeneousEllipsoidalSoundModel(sound_speed)
    # ellip_model = model
    # Calculate detection times using ellipsoidal model
    base_time = datetime.datetime(2020, 6, 1, 12, 0, 0)
    detection_times = []
    
    print(f"Source: {source}")
    for i, sensor in enumerate(sensors):
        distance = ellip_model.get_distance(source, sensor)
        travel_time = distance / sound_speed
        detection_time = base_time + datetime.timedelta(seconds=travel_time)
        detection_times.append(detection_time)
        print(f"  Capteur {i} à {sensor}: distance={distance:.1f}m")
    
    # Localize with ellipsoidal model
    result_ellip = ellip_model.localize_common_source(sensors, detection_times, )
    
    print(f"\nModèle ellipsoïdal:")
    print(f"  Position: [{result_ellip.x[1]:.6f}, {result_ellip.x[2]:.6f}]")
    print(f"  Erreur: {np.linalg.norm(np.array(result_ellip.x[1:]) - np.array(source)):.6f}")




def test_original_case():
    """Test your exact original case"""
    print("\n=== Test cas original ===")
    
    sound_speed = 1500  # Assumé
    model = HomogeneousEllipsoidalSoundModel(sound_speed)
    
    # Your exact configuration
    detection_pos_1 = [5, 0]
    detection_pos_2 = [-5, 0] 
    detection_pos_3 = [0, 5]
    detection_pos_4 = [0, -5]
    detection_positions = [detection_pos_1, detection_pos_2, detection_pos_3, detection_pos_4]
    
    detection_time_1 = datetime.datetime(2020, 1, 1, 0, 0, 0)
    detection_time_2 = detection_time_1
    detection_time_3 = detection_time_1 
    detection_time_4 = detection_time_1
    detection_times = [detection_time_1, detection_time_2, detection_time_3, detection_time_4]
    
    result = model.localize_common_source(detection_positions, detection_times)
    
    print(f"Position trouvée: [{result.x[1]:.6f}, {result.x[2]:.6f}]")
    print(f"t0 trouvé: {result.x[0]:.3f} s")
    print(f"Coût: {result.cost:.2f}")
    
    # Calculate expected values
    expected_position = [0.0, 0.0]  # Center
    ref_distance = model.get_distance(expected_position, detection_positions[0])
    expected_t0 = -ref_distance / sound_speed
    
    print(f"Position attendue: {expected_position}")
    print(f"Distance au capteur de référence: {ref_distance:.1f} m")
    print(f"t0 attendu: {expected_t0:.3f} s")


if __name__ == "__main__":
    test_tdoa_localization()
    compare_with_spherical()
    test_original_case()

In [ ]:
from datetime import datetime, timedelta
import numpy as np

# exemple : 3 capteurs (lat, lon)
sensors = [(0.0, 77), (0.0, 83), (3, 77), (-3,83)]
source = (2., 80)   # position émise connue
# model = HomogeneousEllipsoidalSoundModel(sound_speed=1480)

# emission time
emit = datetime.utcnow()

# calcul des temps de détection
detection_times = [emit + timedelta(seconds=model.get_sound_travel_time(source, s, date=emit)) for s in sensors]

res = model.localize_common_source(sensors, detection_times, initial_pos=[0, 2.05, 79])
print(res)
print("Solution (t0, lat, lon) :", res.x)
print("Position estimée (lat, lon):", res.x[1:], "vraie pos:", source)


In [ ]:
def test_localization():
    model = HomogeneousEllipsoidalSoundModel(sound_speed=1500)

    # Position source connue
    true_source = [45.0, 1.0]

    # Positions capteurs
    sensors = [
        [45.1, 1.0],
        [45.0, 1.1],
        [44.9, 1.0],
        [45.0, 0.9]
    ]

    # Calcul des temps d'arrivée théoriques
    arrival_times = []
    for sensor in sensors:
        dist = model.get_distance(true_source, sensor)
        travel_time = dist / 1500
        arrival_times.append(travel_time)

    # Conversion en datetime
    base_time = datetime.now()
    detection_times = [base_time + timedelta(seconds=t) for t in arrival_times]

    # Localisation
    result = model.localize_common_source(sensors, detection_times)

    print(f"Vrai position: {true_source}")
    print(f"Position estimée: {result.x[1:]}")
    print(f"Erreur: {np.linalg.norm(np.array(true_source) - np.array(result.x[1:]))} degrés")
test_localization()

In [ ]:
# Test avec votre modèle ellipsoïdal
sound_model = HomogeneousEllipsoidalSoundModel(sound_speed=1485)  # ou HomogeneousEllipsoidalSoundModel()

# Données de test
true_source = [0.0, 0.0]
sensors = [
    [5, 5],
    [-5, 5.0],
    [5, -5.0],
    [-5, -5.]
]
# Calcul des temps d'arrivée
base_time = datetime.now()
arrival_times = []
for sensor in sensors:
    dist = sound_model.get_distance(true_source, sensor)
    travel_time = dist / 1485  # Vitesse du son假设
    arrival_times.append(travel_time)

detection_times = [base_time + timedelta(seconds=t) for t in arrival_times]

# Estimation Monte Carlo
pos = sound_model.MC_error_estimation(5000, pick_sigma=2, velocity_sigma=2.5,
                                     sensors_positions=sensors, detection_times=detection_times)
pos = np.array(pos)
sensors =np.array(sensors)

plt.subplot(2, 1, 1)
# Visualisation
plt.subplot(2, 1, 1)
plt.scatter(pos[:, 2], pos[:, 1])
plt.scatter(true_source[1], true_source[0], color='red', marker='x', s=100)
plt.scatter(sensors[:,1], sensors[:,0])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Monte Carlo Uncertainty Estimation equature')
plt.show()

true_source = [80, 0.0]
sensors = [
    [85, 5],
    [75, 5.0],
    [85, -5.0],
    [75, -5.]
]

# Calcul des temps d'arrivée
base_time = datetime.now()
arrival_times = []
for sensor in sensors:
    dist = sound_model.get_distance(true_source, sensor)
    travel_time = dist / 1485  # Vitesse du son
    arrival_times.append(travel_time)

detection_times = [base_time + timedelta(seconds=t) for t in arrival_times]

# Estimation Monte Carlo
pos = sound_model.MC_error_estimation(5000, pick_sigma=2, velocity_sigma=2.5,
                                     sensors_positions=sensors, detection_times=detection_times)
pos = np.array(pos)
sensors =np.array(sensors)
plt.subplot(2, 1, 2)
plt.scatter(pos[:, 2], pos[:, 1])
plt.scatter(true_source[1], true_source[0], color='red', marker='x', s=100)
plt.scatter(sensors[:,1], sensors[:,0])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Monte Carlo Uncertainty Estimation pole nord')
plt.show()

# Statistiques
print(f"Mean position: {np.mean(pos[:, 1:], axis=0)}")
print(f"True position: {true_source}")
print(f"Standard deviation: {np.std(pos[:, 1:], axis=0)}")